<a href="https://colab.research.google.com/github/harshitadd/Celestini/blob/master/November_FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install firebase-admin 

In [0]:
import firebase_admin
from firebase_admin import credentials, db, storage 
import cv2 
import math 
import numpy as np
import matplotlib.pyplot as plt 

In [0]:

cred = credentials.Certificate("/content/celestinihdr-2-firebase-adminsdk-x7gew-beb69daef0.json")
app = firebase_admin.initialize_app(cred, {
    'storageBucket': 'celestinihdr-2.appspot.com',
}, name='novemeber')

In [0]:
bucket = storage.bucket(app = app)
blobs = bucket.list_blobs()
for blob in blobs:
  print(blob.name)

In [0]:
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

anandvihar_hdr=[]
wazirpur_hdr=[]
wazirpur_jpeg={}
ctr=0
for blob in blobs:
  if(blob.name[7:13]=='static' and blob.name[14]=='P' and blob.name[27]=='j' and int(blob.name[43:45])<18 and int(blob.name[43:45])>6):
    blob.download_to_filename('/content/' + "wazirpur_jpeg" + str(blob.name.split('/')[-1])+ '.jpeg')
    name = "wazirpur_jpeg" + str(blob.name.split('/')[-1])+ '.jpeg' 
    ctr+=1
    print(ctr)
    im = cv2.imread(name)
    im = cv2.resize(im,(256,256))
    wazirpur_jpeg[blob.name]=im
  # if(blob.name[7:13]=='static' and blob.name[14]=='P' and blob.name[27]=='h' and int(blob.name[42:44])<18 and int(blob.name[42:44])>6): 
  #   blob.download_to_filename('/content/' + "wazirpur_hdr" + str(blob.name.split('/')[-1])+ '.jpeg')
  #   name = "wazirpur_hdr" + str(blob.name.split('/')[-1])+ '.jpeg' 
  #   ctr+=1
  #   print(ctr)
  #   im = cv2.imread(name)
  #   im = cv2.resize(im,(256,256))
  #   wazirpur_hdr.append(im)
  # if(blob.name[7:13]=='static' and blob.name[14]=='H' and blob.name[21]=='h'):
  #    blob.download_to_filename('/content/' + "anandvihar_hdr" + str(blob.name.split('/')[-1])+ '.jpeg')
  #    name = "anandvihar_hdr" + str(blob.name.split('/')[-1])+ '.jpeg' 
  #    ctr+=1
  #    print(ctr)
  #    im = cv2.imread(name)
  #    im = cv2.resize(im,(256,256))
  #    anandvihar_hdr.append(im)
     
    
    

In [37]:
#print(len(wazirpur_hdr))
print(len(wazirpur_jpeg))
#print(len(anandvihar_hdr))

287


IMAGE PARAMETERS 

In [0]:
def trans_deg(im):
  B=np.zeros((256,256))
  D=np.zeros((256,256))
  C=np.zeros((256,256))
  # 3rd channel or im[2] has been split as b,g,r 

  b,g,r = cv2.split(im)

  for i in range(b.shape[0]):
    for j in range(b.shape[1]):
      D[i][j] = min(b[i][j], g[i][j], r[i][j])
      B[i][j] = max(b[i][j], g[i][j], r[i][j])
      C[i][j] = B[i][j] - D[i][j]

  d = np.sum(D)/float(np.size(D))
  b = np.sum(B)/float(np.size(B))
  c = b-d 
  A = (0.33)*max(np.ndarray.flatten(B)) + (0.66)*b
  
  x1=((A - d)/float(A)) 
  x2=(c/float(A))
  u = 10.127489
  v= -8.336512
  s = 0.13606234
  
  w = math.exp(-0.5*(u*x1 + v*x2)+s)
  
  if(w<=0.1):
      w=0
  elif(w<=0.3):
    w=1
  elif(w<=0.5):
    w=2
  elif(w<=0.7):
    w=3
  elif(w<=0.8):
    w=4
  elif(w<=0.9):
    w=5
    
  return w

In [0]:
def entropy(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hist = cv2.calcHist( [image.astype('float32')],
              [0], 
              None,
               [256], 
              [0,256] )
  h_norm = 0
  h_norm = cv2.normalize(hist, h_norm)
  ent = 0
  for p in h_norm:
    try:
      ent += p*math.log(p, 2)
    except:
      pass
    
  return (-1 * ent[0])

In [0]:
def contrast(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  s = 0
  ss = 0
  mean = np.mean(image)
  x = (-1 * mean * (np.ones(image.shape)))
  ss = np.square(np.add(image, x))
  cont = np.sqrt(np.sum(ss)/float(image.size))
  return cont

In [47]:
feat=[]
for i in wazirpur_jpeg:
  feat.append(i[43:45])
  feat.append(entropy(im))
  feat.append(contrast(im))
  feat.append(trans_deg(im))
  
  break

TypeError: ignored

WEATHER PARAMETERS 

In [0]:
cred = credentials.Certificate('/content/credentials_weather.json')

##The weather data after 1st July 

# Initialize the app with a service account, granting admin privileges
new = firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://celestini-bd493.firebaseio.com/'
}, name ='w')

In [0]:

ref = db.reference(app=new)
df = ref.get()


In [0]:
temp_weather = {}

for key in df:
  try:
    l=[]
    l.append(df[key]['main']['humidity'])
    l.append(df[key]['main']['pressure'])
    l.append(df[key]['main']['temp'])
    l.append(df[key]['visibility'])
    l.append(df[key]['wind']['deg'])
    l.append(df[key]['wind']['speed'])
    l.append(df[key]['clouds']['all'])
    k = df[key]['weather'][0]['description']
    if(k=='haze'):
      l.append(0)
    elif(k=='mist'):
      l.append(1)
    elif(k=='drizzle'):
      l.append(2)
    elif(k=='dust'):
      l.append(3)
    temp_weather[key]=l
  except:
    print(key)
    